In [1]:
# initialize findspark for local
import findspark
findspark.init('/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7')

In [2]:
import configparser
from datetime import datetime
# import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [3]:
# Configure the necessary Spark environment
import os
import sys
import boto3

In [4]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.1 pyspark-shell'

In [5]:
pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, "/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip"))

# Initialize PySpark
exec(open(os.path.join(spark_home, "/Users/johnrick/Downloads/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py")).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
SparkSession available as 'spark'.


In [ ]:
# spark-submit --packages org.apache.hadoop:hadoop-aws:2.7.0

In [ ]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:2.7.0'

In [ ]:
#  --packages com.amazonaws:aws-java-sdk-pom:1.11.760,org.apache.hadoop:hadoop-aws:2.7.0 --conf spark.hadoop.fs.s3a.endpoint=s3.us-west-2.amazonaws.com

In [6]:
from pyspark.sql.types import StructType as R, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType, StructType, LongType

In [7]:
# config = configparser.ConfigParser()
# config.read('dl.cfg')

# os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [8]:
# # store s3 bucket location
# input_data = "s3://udacity-dend/song_data"

# song_data = input_data + "/*/*/*/*.json"

# Get data

## create boto3 session

In [9]:
target_region = "us-west-2"

In [ ]:
s3 = boto3.resource('s3',
                    region_name=target_region,
                   aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'],
                   aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'])

In [10]:
import logging
import boto3
from botocore.exceptions import ClientError

In [11]:
def create_bucket(bucket_name, region=None, acl="private"):
    """
    Create an S3 bucket in a specified region
    
    Args:
        bucket_name: Bucket to create
        region: String region to create bucket in, e.g., 'us-west-2'
        acl: Canned access control list to apply to the bucket. 'public-read'
            makes sure everything posted is publicly readable
    """
    
    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration=location,
                ACL=acl
            )
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [12]:
create_bucket('ud1-s3datalake', target_region, acl='public-read')

True

In [13]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [14]:
#test on 24 items
song_data = "s3a://udacity-dend/song_data/A/A/A/*.json"

In [ ]:
#song_data = "s3a://udacity-dend/song_data/*/*/*/*.json"

In [15]:
song_schema = StructType([
        StructField("artist_id", StringType(), False),
        StructField("artist_latitude", StringType(), True),
        StructField("artist_longitude", StringType(), True),
        StructField("artist_location", StringType(), True),
        StructField("artist_name", StringType(), False),
        StructField("song_id", StringType(), False),
        StructField("title", StringType(), False),
        StructField("duration", DoubleType(), False),
        StructField("year", IntegerType(), False)
])

In [16]:
df = spark.read.json(song_data, schema=song_schema)

In [17]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[artist_id: string, artist_latitude: string, artist_longitude: string, artist_location: string, artist_name: string, song_id: string, title: string, duration: double, year: int]>

In [18]:
p_df = df.toPandas()

In [19]:
len(p_df)

24

In [20]:
p_df.head()

,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,ARTC1LV1187B9A4858,51.4536,-0.01802,"Goldsmith's College, Lewisham, Lo",The Bonzo Dog Band,SOAFBCP12A8C13CC7D,King Of Scurf (2007 Digital Remaster),301.40036,1972
1,ARA23XO1187B9AF18F,40.57885,-74.21956,"Carteret, New Jersey",The Smithereens,SOKTJDS12AF72A25E5,Drown In My Own Tears (24-Bit Digitally Remast...,192.52200,0
2,ARSVTNL1187B992A91,51.50632,-0.12714,"London, England",Jonathan King,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),129.85424,2001
3,AR73AIO1187B9AD57B,37.77916,-122.42005,"San Francisco, CA",Western Addiction,SOQPWCR12A6D4FB2A3,A Poor Recipe For Civic Cohesion,118.07302,2005
4,ARXQBR11187B98A2CC,None,None,"Liverpool, England",Frankie Goes To Hollywood,SOBRKGM12A8C139EF6,Welcome to the Pleasuredome,821.05424,1985


## songs_table insert

In [21]:
# extract columns to create songs table
songs_table = df.select("song_id",
                       "title", 
                       "artist_id",
                       "year",
                       "duration").dropDuplicates()

In [30]:
#write to our own s3 bucket
songs_table.write.partitionBy(['year','artist_id']).format("parquet").mode("overwrite").save("s3a://ud1-s3datalake/Songs")
#parquet("s3a://ud-s3datalake/Songs")

Py4JJavaError: An error occurred while calling o121.save.
: com.amazonaws.services.s3.model.AmazonS3Exception: Status Code: 400, AWS Service: Amazon S3, AWS Request ID: A6C13AFF39CE4B13, AWS Error Code: null, AWS Error Message: Bad Request, S3 Extended Request ID: SWw4kUy1GkEUKVHEOEvBC6Xp3xLTwerbPlha2swdASbJ66aT62ZdljvDzKEHSoDx9090pIlIrs4=
	at com.amazonaws.http.AmazonHttpClient.handleErrorResponse(AmazonHttpClient.java:798)
	at com.amazonaws.http.AmazonHttpClient.executeHelper(AmazonHttpClient.java:421)
	at com.amazonaws.http.AmazonHttpClient.execute(AmazonHttpClient.java:232)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3528)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.h

In [24]:
# #format 2
# songs_table.write.parquet(
#         "s3a://ud1-s3datalake/songs_table.parquet",
#         mode="overwrite",
#         partitionBy=["year", "artist_id"])

### Test If we are loading correctly

In [ ]:
### Function for quality check
def validate_table_insertion(table, id):
    """
    Read the parquet file into dataframe, convert spark df to sql table
    Check the table using sql queries
    """
    ### load table
    table_name = os.path.join(input_path, table)
    parquetFile = spark.read.parquet(table_name)
    ### print count
    print('\033[1m' + f"Validate insertion from table {table}:" + '\033[0m')
    print("Dimension of the table:", parquetFile.count())
    
    ### convert df to sql table
    parquetFile.createOrReplaceTempView(table)
    
    ### count unique
    unique = spark.sql(f"SELECT COUNT(DISTINCT {id}) count FROM {table};")
    print("Number of unique id: ", unique.select('count').collect()[0]['count'])
    ### check null id
    
    null_id = spark.sql(f"SELECT COUNT(*) count FROM {table} WHERE {id} = NULL;")    
    print("Number of null id: ", null_id.select('count').collect()[0]['count'])
    
    ### print some examples
    print("Some examples from the table:")
    examples = spark.sql(f"SELECT * FROM {table} LIMIT 5;")
    examples.show()

In [ ]:
input_path

In [ ]:
input_path = 'output' # location of all the tables
tables = ['songs', 'artists', 'users', 'time', 'songplays']
ids = ["song_id", "artist_id", "user_id", "start_time", "start_time"]

### Perform all the check:
for i in range(0, len(tables)):
    validate_table_insertion(tables[i], ids[i])

## Artists Table

In [27]:
# extract column to get artists_table
artists_table = df.select("artist_id",
                         col("artist_name").alias("name"),
                         col("artist_location").alias("location"),
                         col("artist_latitude").alias("latitude"),
                         col("artist_longitude").alias("longitude")).dropDuplicates()

In [29]:
# write artists table to parquet files
artists_table.write.format("parquet").mode("overwrite").save("s3a://ud1-s3datalake/Artists")


In [ ]:
# #write to our own s3 bucket
# songs_table.write.partitionBy(['year','artist_id']).format("parquet").mode("overwrite").save("s3a://ud1-s3datalake/Songs")
# #parquet("s3a://ud-s3datalake/Songs")

# Process Log Data

In [ ]:
# process_log_data(spark, input_data, output_data)


In [31]:
log_schema = StructType([
        StructField("artist", StringType(), True),
        StructField("auth", StringType(), False),
        StructField("firstName", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("itemInSession", IntegerType(), False),
        StructField("lastName", StringType(), True),
        StructField("length", DoubleType(), True),
        StructField("level", StringType(), False),
        StructField("location", StringType(), True),
        StructField("method", StringType(), False),
        StructField("page", StringType(), False),
        StructField("registration", DoubleType(), True),
        StructField("sessionId", IntegerType(), False),
        StructField("song", StringType(), True),
        StructField("status", IntegerType(), False),
        StructField("ts", DoubleType(), False),
        StructField("userAgent", StringType(), True),
        StructField("userId", StringType(), True)
    ])

In [36]:
# get filepath to log data file
log_data = "s3a://udacity-dend/log_data/*/*/*.json"

In [37]:
# read log data file
df = spark.read.json(log_data, schema = log_schema)

In [38]:
# filter by actions for song plays
df = df.where(df.page == "NextSong")

## users_table insert

In [43]:
users_table = df.select(col("userId").alias("user_id"),
                       col("firstName").alias("first_name"),
                       col("lastName").alias("last_name"),
                       "gender",
                       "level").dropDuplicates()

In [44]:
users_table.write.format("parquet").mode("overwrite").save("s3a://ud1-s3datalake/Users")

## Time Stamp

In [46]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df = df.withColumn("timestamp", get_timestamp(df.ts))

In [47]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))
df = df.withColumn("datetime", get_datetime(df.ts))

In [48]:
# extract columns to create time table
time_table = df.select(
    df.timestamp.alias('start_time'),
    hour(df.datetime).alias('hour'),
    dayofmonth(df.datetime).alias('day'),
    weekofyear(df.datetime).alias('week'),
    month(df.datetime).alias('month'),
    year(df.datetime).alias('year'),
    date_format(df.datetime, 'u').alias('weekday')).dropDuplicates()

In [54]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").format("parquet").save("s3a://ud1-s3datalake/Time")

In [53]:
# extract columns to create time table
time_table = df.select(
    df.timestamp.alias('start_time'),
    hour(df.datetime).alias('hour'),
    dayofmonth(df.datetime).alias('day'),
    weekofyear(df.datetime).alias('week'),
    month(df.datetime).alias('month'),
    year(df.datetime).alias('year'),
    date_format(df.datetime, 'u').alias('weekday')).dropDuplicates().limit(10)

# songplays_table

In [ ]:
# read in song data to use for songplays table
song_df = 

In [ ]:
# extract columns from joined song and log datasets to create songplays table 
songplays_table = 

In [ ]:
# write songplays table to parquet files partitioned by year and month
songplays_table